In [2]:
import requests
import pandas as pd
from datetime import datetime

BINANCE_URL = "https://api.binance.com/api/v3/klines"


# ------------------------ UTILIDADES DE DATOS ------------------------ #

def fetch_klines_full(symbol: str, interval: str, start: datetime) -> pd.DataFrame:
    """
    Descarga TODO el histórico desde `start` hasta hoy para un símbolo/intervalo,
    usando paginación de Binance (interval: '1d', '1w', etc.)
    """
    all_rows = []
    start_time = int(start.timestamp() * 1000)

    while True:
        params = {
            "symbol": symbol,
            "interval": interval,
            "startTime": start_time,
            "limit": 1000
        }
        resp = requests.get(BINANCE_URL, params=params, timeout=10)
        resp.raise_for_status()
        data = resp.json()
        if not data:
            break
        all_rows.extend(data)
        if len(data) < 1000:
            break
        last_open_time = data[-1][0]
        start_time = last_open_time + 1

    if not all_rows:
        raise ValueError(f"Sin datos para {symbol} {interval}")

    df = pd.DataFrame(all_rows, columns=[
        "open_time","open","high","low","close","volume",
        "close_time","qav","num_trades",
        "taker_buy_base","taker_buy_quote","ignore"
    ])

    df["date"] = pd.to_datetime(df["open_time"], unit="ms")
    df["close"] = df["close"].astype(float)
    df["volume"] = df["volume"].astype(float)
    df["taker_buy_base"] = df["taker_buy_base"].astype(float)
    df["taker_sell_base"] = df["volume"] - df["taker_buy_base"]
    df = df.sort_values("date").reset_index(drop=True)
    return df


def make_signal_series(
        df: pd.DataFrame,
        lookback: int = 7,
        sell_pressure_threshold: float = 1.3,
        drop_threshold_pct: float = 3.0,
        volume_mult_threshold: float = 1.3
) -> pd.Series:
    """
    Genera una serie booleana 'signal' por fila, a partir de lookback velas previas.

    Señal = volumen alto *y* (caída de precio o presión vendedora alta)
    """
    n = len(df)
    signal = [False] * n

    for i in range(lookback, n):
        prev = df.iloc[i - lookback:i]
        last = df.iloc[i]

        prev_avg_close = prev["close"].mean()
        prev_avg_vol = prev["volume"].mean()

        last_close = last["close"]
        last_vol = last["volume"]

        taker_buy = last["taker_buy_base"]
        taker_sell = last["taker_sell_base"]

        sell_pressure = (taker_sell + 1e-9) / (taker_buy + 1e-9)
        drop_pct = (prev_avg_close - last_close) / prev_avg_close * 100 if prev_avg_close > 0 else 0
        vol_mult = last_vol / prev_avg_vol if prev_avg_vol > 0 else 1

        cond_drop = drop_pct >= drop_threshold_pct
        cond_sell = sell_pressure >= sell_pressure_threshold
        cond_vol = vol_mult >= volume_mult_threshold

        signal[i] = bool(cond_vol and (cond_drop or cond_sell))

    return pd.Series(signal, index=df.index, name="signal")


# ------------------------ BACKTEST 3 ESTRATEGIAS ------------------------ #

def backtest_daily_weekly_combined(
        start_date: str,
        end_date: str,
        monthly_budget: float = 2000.0,
        weight_btc: float = 0.7,
        weight_eth: float = 0.3,
        lookback: int = 7,
        sell_pressure_threshold: float = 1.3,
        drop_threshold_pct: float = 3.0,
        volume_mult_threshold: float = 1.3,
        combine_mode: str = "or"  # "or" o "and"
):
    """
    Compara 3 estrategias:

      1) DAILY:
         - Usa solo señal DIARIA.
         - Inversión máx. por día = monthly_budget / 30.

      2) WEEKLY:
         - Usa solo señal SEMANAL.
         - Inversión máx. por semana = monthly_budget / 4.

      3) DAILY + WEEKLY COMBINED:
         - Señal diaria + señal semanal mapeada a cada día.
         - Señal combinada:
              if combine_mode == "or": daily OR weekly
              if combine_mode == "and": daily AND weekly
         - Inversión máx. por día = monthly_budget / 30.

    En todos los casos:
      - Se invierte solo si la señal del período (día o semana) está ON.
      - Se reparte el presupuesto 70% BTC / 30% ETH.
    """

    start_dt = datetime.fromisoformat(start_date)
    end_dt = datetime.fromisoformat(end_date)

    # ----- Datos diarios BTC/ETH -----
    print("Descargando datos diarios BTCUSDT y ETHUSDT...")
    btc_d = fetch_klines_full("BTCUSDT", "1d", start_dt)
    eth_d = fetch_klines_full("ETHUSDT", "1d", start_dt)

    btc_d = btc_d[btc_d["date"] <= end_dt].reset_index(drop=True)
    eth_d = eth_d[eth_d["date"] <= end_dt].reset_index(drop=True)

    # Alineamos fechas
    all_dates = pd.date_range(
        start=max(btc_d["date"].min(), eth_d["date"].min()),
        end=min(btc_d["date"].max(), eth_d["date"].max()),
        freq="D"
    )
    btc_d = btc_d.set_index("date").reindex(all_dates).ffill()
    eth_d = eth_d.set_index("date").reindex(all_dates).ffill()
    btc_d.index.name = "date"
    eth_d.index.name = "date"

    # Señales diarias
    print("Calculando señal diaria...")
    btc_d["signal_daily"] = make_signal_series(
        btc_d,
        lookback=lookback,
        sell_pressure_threshold=sell_pressure_threshold,
        drop_threshold_pct=drop_threshold_pct,
        volume_mult_threshold=volume_mult_threshold,
    )
    eth_d["signal_daily"] = make_signal_series(
        eth_d,
        lookback=lookback,
        sell_pressure_threshold=sell_pressure_threshold,
        drop_threshold_pct=drop_threshold_pct,
        volume_mult_threshold=volume_mult_threshold,
    )

    # ----- Datos semanales BTC/ETH -----
    print("Descargando datos semanales BTCUSDT y ETHUSDT...")
    btc_w = fetch_klines_full("BTCUSDT", "1w", start_dt)
    eth_w = fetch_klines_full("ETHUSDT", "1w", start_dt)

    btc_w = btc_w[btc_w["date"] <= end_dt].reset_index(drop=True)
    eth_w = eth_w[eth_w["date"] <= end_dt].reset_index(drop=True)

    # Señales semanales (a nivel vela semanal)
    print("Calculando señal semanal...")
    btc_w["signal_weekly"] = make_signal_series(
        btc_w,
        lookback=lookback,
        sell_pressure_threshold=sell_pressure_threshold,
        drop_threshold_pct=drop_threshold_pct,
        volume_mult_threshold=volume_mult_threshold,
    )
    eth_w["signal_weekly"] = make_signal_series(
        eth_w,
        lookback=lookback,
        sell_pressure_threshold=sell_pressure_threshold,
        drop_threshold_pct=drop_threshold_pct,
        volume_mult_threshold=volume_mult_threshold,
    )

    # Mapear señal semanal a calendario diario (ffill) para estrategia combinada
    all_dates = btc_d.index  # ya es el rango diario común
    btc_weekly_signal_daily = (
        btc_w.set_index("date")["signal_weekly"]
        .reindex(all_dates)
        .ffill()
        .fillna(False)
        .astype(bool)
    )
    eth_weekly_signal_daily = (
        eth_w.set_index("date")["signal_weekly"]
        .reindex(all_dates)
        .ffill()
        .fillna(False)
        .astype(bool)
    )

    # Presupuestos por período
    daily_budget = monthly_budget / 30.0      # para DAILY y COMBINED
    weekly_budget = monthly_budget / 4.0      # para WEEKLY

    # ----------------- ESTRATEGIA 1: DAILY ----------------- #
    btc_units_daily = 0.0
    eth_units_daily = 0.0
    invested_daily = 0.0
    buy_days_daily = 0

    for i, date in enumerate(all_dates):
        if i < lookback:
            continue

        sig_btc = bool(btc_d.loc[date, "signal_daily"])
        sig_eth = bool(eth_d.loc[date, "signal_daily"])

        day_signal = sig_btc or sig_eth
        if not day_signal:
            continue

        amount_btc = daily_budget * weight_btc
        amount_eth = daily_budget * weight_eth

        price_btc = btc_d.loc[date, "close"]
        price_eth = eth_d.loc[date, "close"]

        btc_units_daily += amount_btc / price_btc
        eth_units_daily += amount_eth / price_eth
        invested_daily += daily_budget
        buy_days_daily += 1

    # ----------------- ESTRATEGIA 2: WEEKLY ----------------- #
    btc_units_weekly = 0.0
    eth_units_weekly = 0.0
    invested_weekly = 0.0
    buy_weeks = 0

    for i in range(lookback, len(btc_w)):
        date = btc_w.loc[i, "date"]
        sig_btc = bool(btc_w.loc[i, "signal_weekly"])
        sig_eth = bool(eth_w.loc[i, "signal_weekly"])

        week_signal = sig_btc or sig_eth
        if not week_signal:
            continue

        amount_btc = weekly_budget * weight_btc
        amount_eth = weekly_budget * weight_eth

        price_btc = btc_w.loc[i, "close"]
        price_eth = eth_w.loc[i, "close"]

        btc_units_weekly += amount_btc / price_btc
        eth_units_weekly += amount_eth / price_eth
        invested_weekly += weekly_budget
        buy_weeks += 1

    # ----------------- ESTRATEGIA 3: DAILY + WEEKLY COMBINED ----------------- #
    btc_units_comb = 0.0
    eth_units_comb = 0.0
    invested_comb = 0.0
    buy_days_comb = 0

    for i, date in enumerate(all_dates):
        if i < lookback:
            continue

        d_btc = bool(btc_d.loc[date, "signal_daily"])
        d_eth = bool(eth_d.loc[date, "signal_daily"])
        w_btc = bool(btc_weekly_signal_daily.loc[date])
        w_eth = bool(eth_weekly_signal_daily.loc[date])

        if combine_mode == "and":
            c_btc = d_btc and w_btc
            c_eth = d_eth and w_eth
        else:  # "or"
            c_btc = d_btc or w_btc
            c_eth = d_eth or w_eth

        day_signal_comb = c_btc or c_eth
        if not day_signal_comb:
            continue

        amount_btc = daily_budget * weight_btc
        amount_eth = daily_budget * weight_eth

        price_btc = btc_d.loc[date, "close"]
        price_eth = eth_d.loc[date, "close"]

        btc_units_comb += amount_btc / price_btc
        eth_units_comb += amount_eth / price_eth
        invested_comb += daily_budget
        buy_days_comb += 1

    # ----- Valor final con precios del último día ----- #
    final_btc_price = btc_d["close"].iloc[-1]
    final_eth_price = eth_d["close"].iloc[-1]

    value_daily = btc_units_daily * final_btc_price + eth_units_daily * final_eth_price
    value_weekly = btc_units_weekly * final_btc_price + eth_units_weekly * final_eth_price
    value_comb = btc_units_comb * final_btc_price + eth_units_comb * final_eth_price

    profit_daily = value_daily - invested_daily
    profit_weekly = value_weekly - invested_weekly
    profit_comb = value_comb - invested_comb

    pct_daily = (profit_daily / invested_daily * 100) if invested_daily > 0 else 0
    pct_weekly = (profit_weekly / invested_weekly * 100) if invested_weekly > 0 else 0
    pct_comb = (profit_comb / invested_comb * 100) if invested_comb > 0 else 0

    # ----- Resultados ----- #
    print("\n================= RESULTADOS BACKTEST =================")
    print(f"Período: {all_dates[0].date()} → {all_dates[-1].date()}")
    print(f"Presupuesto nominal: {monthly_budget:.2f} USD/mes, 70% BTC / 30% ETH")
    print("-------------------------------------------------------")

    print("\n--- Estrategia 1: DAILY (solo señal diaria) ---")
    print(f"Días con compra:      {buy_days_daily}")
    print(f"Total invertido:      ${invested_daily:,.2f}")
    print(f"Valor final cartera:  ${value_daily:,.2f}")
    print(f"Ganancia:             ${profit_daily:,.2f} ({pct_daily:.2f}%)")

    print("\n--- Estrategia 2: WEEKLY (solo señal semanal) ---")
    print(f"Semanas con compra:   {buy_weeks}")
    print(f"Total invertido:      ${invested_weekly:,.2f}")
    print(f"Valor final cartera:  ${value_weekly:,.2f}")
    print(f"Ganancia:             ${profit_weekly:,.2f} ({pct_weekly:.2f}%)")

    print("\n--- Estrategia 3: DAILY + WEEKLY COMBINED ---")
    print(f"Modo combinación:     {combine_mode.upper()}")
    print(f"Días con compra:      {buy_days_comb}")
    print(f"Total invertido:      ${invested_comb:,.2f}")
    print(f"Valor final cartera:  ${value_comb:,.2f}")
    print(f"Ganancia:             ${profit_comb:,.2f} ({pct_comb:.2f}%)")

    # comparativa básica
    print("\n>>> Comparación % retorno:")
    print(f"DAILY:   {pct_daily:.2f}%")
    print(f"WEEKLY:  {pct_weekly:.2f}%")
    print(f"COMBINED:{pct_comb:.2f}%")

    return {
        "invested_daily": invested_daily,
        "value_daily": value_daily,
        "profit_daily": profit_daily,
        "pct_daily": pct_daily,
        "buy_days_daily": buy_days_daily,

        "invested_weekly": invested_weekly,
        "value_weekly": value_weekly,
        "profit_weekly": profit_weekly,
        "pct_weekly": pct_weekly,
        "buy_weeks": buy_weeks,

        "invested_comb": invested_comb,
        "value_comb": value_comb,
        "profit_comb": profit_comb,
        "pct_comb": pct_comb,
        "buy_days_comb": buy_days_comb,
    }


# ------------------------ EJEMPLO DE USO ------------------------ #

if __name__ == "__main__":
    results = backtest_daily_weekly_combined(
        start_date="2020-01-01",
        end_date="2025-12-01",
        monthly_budget=2000.0,
        weight_btc=0.7,
        weight_eth=0.3,
        lookback=7,
        sell_pressure_threshold=1.3,
        drop_threshold_pct=3.0,
        volume_mult_threshold=1.3,
        combine_mode="or"   # probá también "and"
    )


Descargando datos diarios BTCUSDT y ETHUSDT...
Calculando señal diaria...
Descargando datos semanales BTCUSDT y ETHUSDT...
Calculando señal semanal...

================= RESULTADOS BACKTEST =================
Período: 2020-01-02 → 2025-12-01
Presupuesto nominal: 2000.00 USD/mes, 70% BTC / 30% ETH
-------------------------------------------------------

--- Estrategia 1: DAILY (solo señal diaria) ---
Días con compra:      202
Total invertido:      $13,466.67
Valor final cartera:  $36,054.18
Ganancia:             $22,587.52 (167.73%)

--- Estrategia 2: WEEKLY (solo señal semanal) ---
Semanas con compra:   31
Total invertido:      $15,500.00
Valor final cartera:  $52,152.22
Ganancia:             $36,652.22 (236.47%)

--- Estrategia 3: DAILY + WEEKLY COMBINED ---
Modo combinación:     OR
Días con compra:      353
Total invertido:      $23,533.33
Valor final cartera:  $68,622.81
Ganancia:             $45,089.47 (191.60%)

>>> Comparación % retorno:
DAILY:   167.73%
WEEKLY:  236.47%
COMBINED: